Trong notebook này, ta sẽ trình bày hai cách khác nhau để viết hàm kNN, và so sánh tốc độ thực thi giữa chúng.

### Bước 1: Chuẩn bị 

Trước tiên, sẽ là hàm tính khoảng cách từ một điểm đến từng điểm trong một tập hợp.

In [2]:
import numpy as np
from time import time     # for comparing running time 

Lưu ý: Các điểm dữ liệu (quan sát) được sắp xếp theo các hàng, suy ra số hàng là số điểm dữ liệu, số cột là số feature của dữ liệu.

In [3]:
d, N = 1000, 10000           # dimension - feature (1000 - row), number of training points (10000 - column)
X = np.random.rand(N, d)     # N d-dimensional points (N x d matrix)
z = np.random.rand(d)

#### 1. Hàm tính bình phương khoảng cách Euclide giữa z và x 

In [4]:
# naively compute square distance between two vectors 
def dist_pp(z, x): 
    d = z - x.reshape(z.shape)    # force z and x to have the same dimensions (not necessary)
    return np.sum(d * d)          # square distance 

#### 2.1. Hàm tính bình phương khoảng cách giữa z và mỗi hàng của X (naive)

In [9]:
# from one point to each point in a set, naive 
def dist_ps_naive(z, x): 
    """Distance from one point to (each point in) a set, normally computed.
    """
    N = X.shape[0]                 # get the number of rows of X (number of points/observations)
    res = np.zeros((1, N))         # create a 1 x N zeros matrix to obtain distances 
    
    for i in range(N): 
        res[0][i] = dist_pp(z, X[i])
    
    return res 

#### 2.2. Tính bình phương khoảng cách giữa z và mỗi hàng của X (nhanh)

Ta dựa vào công thức sau: 
\begin{equation}
\lVert \mathbf{z} - \mathbf{x}_i \rVert_{2}^{2} = (\mathbf{z} - \mathbf{x}_i)^T(\mathbf{z} - \mathbf{x}_i) = \lVert \mathbf{z} \rVert_{2}^{2} + \lVert \mathbf{x}_i \rVert_{2}^{2} - 2\mathbf{x}_{i}^T\mathbf{z} \quad \quad (1)
\end{equation}

Từ công thức trên, vì z cố định, nên chỉ cần tính số hạng thứ hai và thứ ba là được. Ngoài ra, số hạng thứ hai có thể được tính toán và lưu trữ từ trước.

In [10]:
# from one point to each point in a set, fast (vectorization + broadcasting)
def dist_ps_fast(z, X): 
    """Distance from one point to (each point in) a set, fast computed.
    """
    X2 = np.sum(X*X, 1)  # (vectorization) square of l2 norm of each X[i], can be precomputed 
                         # (lay tong cua tung hang cua MT) => tra ve mot vector hang 
    z2 = np.sum(z*z)     # square of l2 norm of z 
    return X2 + z2 - 2*X.dot(z)    # z2 can be ignored 

So sánh thời gian chạy giữa hàm 2.1 và hàm 2.2:

In [8]:
t1 = time()
D1 = dist_ps_naive(z, X)
print('naive point2set, running time:', time() - t1, 's')

t1 = time()
D2 = dist_ps_fast(z, X)
print('fast point2set , running time:', time() - t1, 's')
print('Result difference:', np.linalg.norm(D1 - D2))    # sai so sinh ra tu dau nhi???

naive point2set, running time: 0.21126461029052734 s
fast point2set , running time: 0.5729484558105469 s
Result difference: 1.0129738073044617e-11


In [11]:
D1

array([[165.55245599, 163.95764607, 162.23317194, ..., 174.01198993,
        175.49877341, 161.27242551]])

In [12]:
D2

array([165.55245599, 163.95764607, 162.23317194, ..., 174.01198993,
       175.49877341, 161.27242551])

Nhận xét: Kết quả chỉ ra rằng hàm dist_ps_fast(z, X) chạy nhanh hơn gần gấp đôi so với
hàm dist_ps_naive(z, X).  Tỉ lệ này còn lớn hơn khi kích thước dữ liệu tăng lên
và X2 được tính từ trước.

#### 3. Cải tiến thuật toán 

In [14]:
# naively compute square distance between two vectors 
def dist_pp2(z, x): 
    d = z - x                     # force z and x to have the same dimensions
    return np.sum(d * d)          # square distance 

In [15]:
# from one point to each point in a set, naive 
def dist_ps_naive2(z, x): 
    N = X.shape[0]                 # get the number of rows of X (number of points/observations)
    res = np.zeros(N)              # create a zeros vector (row vector)
    
    for i in range(N): 
        res[i] = dist_pp2(z, X[i])
    
    return res 

In [16]:
t1 = time()
D3 = dist_ps_naive2(z, X)
print('naive point2set, running time:', time() - t1, 's')

naive point2set, running time: 0.09784197807312012 s


In [17]:
D1 == D3  # kiem tra xem hai thuat toan co ra ket qua giong nhau kh

array([[ True,  True,  True, ...,  True,  True,  True]])

### Bước 2: Thực hiện thuật toán kNN

#### C1: Cách đơn giản nhất:

Ý tưởng: Lấy từng điểm trong tập thứ nhất, tính khoảng cách từ điểm này đến tất cả các điểm trong tập thứ hai (sử dụng hàm khoảng cách thông thường).

Sử dụng một vòng
for tính khoảng cách từ từng điểm trong tập thứ nhất đến tất cả các điểm trong
tập thứ hai thông qua hàm **dist_ps_fast(z, X)** ở trên. 

In [4]:
import numpy as np 
from time import time 

In [2]:
# from one point to each point in a set, fast 
def dist_ps_fast(z, X): 
    X2 = np.sum(X*X, 1)  # (vectorization) square of l2 norm of each X[i], can be precomputed 
                         # (lay tong cua tung hang cua MT) 
    z2 = np.sum(z*z)     # square of l2 norm of z 
    return X2 + z2 - 2*X.dot(z)    # z2 can be ignored 

In [6]:
d, N = 1000, 10000            # dimension - feature (1000 - row), number of training points (10000 - column)
X = np.random.rand(N, d)      # N d-dimensional points (N x d matrix)
Z = np.random.randn(100, d)   # 100 diem DL d chieu    (100 x d matrix)

In [7]:
# from each point in one set to each point in another set, half fast 
def dist_ss_0(Z, X): 
    """Distance from (each point in) one set to (each point in) another set, normally computed.
    """
    M, N = Z.shape[0], X.shape[0]    # get the number of data points in each set (M = 100, N = N)
    res = np.zeros((M, N))           # create a M x N (100 x N) matrix to contain distance 
                                     # ptu (i, j) la kcach tu diem thu i (cua Z) den diem thu j (cua N)
    for i in range(M):               # lap qua cac diem o tap Z
        res[i] = dist_ps_fast(Z[i], X)
    
    return res 

#### Cách 2: Sử dụng công thức tính khoảng cách ở dưới 

\begin{equation}
\lVert \mathbf{z} - \mathbf{x}_i \rVert_{2}^{2} = (\mathbf{z} - \mathbf{x}_i)^T(\mathbf{z} - \mathbf{x}_i) = \lVert \mathbf{z} \rVert_{2}^{2} + \lVert \mathbf{x}_i \rVert_{2}^{2} - 2\mathbf{x}_{i}^T\mathbf{z} \quad \quad (1)
\end{equation}

In [8]:
# from each point in one set to each point in another set, fast (vectorization + broadcasting)
def dist_ss_fast(Z, x):
    """Distance from (each point in) one set to (each point in) another set, fast computed.
    """
    X2 = np.sum(X*X, 1)     # square of l2 norm of each ROW of X 
    Z2 = np.sum(Z*Z, 1)     # square of l2 norm of each ROW of Z
    return Z2.reshape(-1, 1) + X2.reshape(1, -1) - 2*Z.dot(X.T)    # vector cột + vector hàng + ma trận 

So sánh kết quả của hai hàm:

In [9]:
t1 = time()
D3 = dist_ss_0(Z, X)
print('half fast set2set running time:', time() - t1, 's')

t1 = time()
D4 = dist_ss_fast(Z, X)
print('fast set2set running time', time() - t1, 's')
print('Result difference:', np.linalg.norm(D3 - D4))

half fast set2set running time: 4.805836915969849 s
fast set2set running time 0.26488399505615234 s
Result difference: 6.173352312305638e-11


Nhận xét: Với lượng dữ liệu lớn thì vectorization + broadcasting sẽ cho kết quả nhanh hơn nhiều.